# Utils

In [1]:
from itertools import product
from math import sqrt
from typing import Tuple, Optional, Callable

import numpy as np
import numpy.linalg as linalg
import pandas as pd
import plotly.graph_objects as go

In [2]:
def find_eig_by_power_method(
        matrix: np.ndarray,
        *,
        eps: float,
        init_x: Optional[np.ndarray] = None,
        limit: int = 10000,
) -> Tuple[float, np.ndarray, int]:
    curr_x = np.ones(matrix.shape[0]) if init_x is None else init_x
    curr_x = curr_x / linalg.norm(curr_x)

    for step in range(limit):
        next_x = matrix.dot(curr_x)
        value = sqrt(next_x.dot(next_x) / curr_x.dot(curr_x))

        if linalg.norm(next_x - value * curr_x) / linalg.norm(curr_x) <= eps:
            return value, next_x, step + 1

        curr_x = next_x / linalg.norm(next_x)

    return value, next_x, step + 1

In [3]:
def find_eig_by_scalar_method(
        matrix: np.ndarray,
        *,
        eps: float,
        init_x: Optional[np.ndarray] = None,
        limit: int = 10000,
) -> Tuple[float, np.ndarray, int]:
    curr_x = np.ones(matrix.shape[0]) if init_x is None else init_x
    
    curr_x, curr_y = curr_x / linalg.norm(curr_x), np.copy(curr_x)
    next_x, next_y = matrix.dot(curr_x), matrix.T.dot(curr_y)

    curr_value = next_x.dot(next_y) / curr_x.dot(next_y)

    for step in range(limit):
        curr_x, curr_y = next_x / linalg.norm(next_x), next_y / linalg.norm(next_y)
        next_x, next_y = matrix.dot(curr_x), matrix.T.dot(curr_y)

        next_value = next_x.dot(next_y) / curr_x.dot(next_y)

        if linalg.norm(next_value - curr_value) <= eps:
            return next_value, next_x, step + 1

        curr_value = next_value

    return next_value, next_x, step + 1

In [4]:
def generate_matrix(element_factory: Callable[[int, int], float], size: int) -> np.ndarray:
    return np.array(
        [
            [element_factory(row, column) for column in range(1, size + 1)]
            for row in range(1, size + 1)
        ],
        dtype=float,
    )


def get_hilbert_matrix(size: int) -> np.ndarray:
    return generate_matrix(lambda row, column: 1 / (row + column - 1), size)

In [5]:
def benchmark_matrix(matrix: np.ndarray, eps: float) -> pd.Series:
    values, vectors = linalg.eig(matrix)
    argmax = abs(values).argmax()

    true_value, true_vector = abs(values)[argmax], vectors[:, argmax]

    power_value, power_vector, power_steps = find_eig_by_power_method(matrix, eps=eps)
    scalar_value, scalar_vector, scalar_steps = find_eig_by_scalar_method(matrix, eps=eps)

    return pd.Series([
        power_steps,
        abs(power_value - true_value),
        linalg.norm(abs(power_vector / linalg.norm(power_vector)) - abs(true_vector)),
        scalar_steps,
        abs(scalar_value - true_value),
        linalg.norm(abs(scalar_vector / linalg.norm(scalar_vector)) - abs(true_vector)),
    ])

In [6]:
def compare_eps_and_steps(data: pd.DataFrame, n: int) -> None:
    data = data[data.n == n]

    fig = go.Figure()

    fig.add_scatter(x=data.eps, y=data.power_steps, name='Степенной метод')
    fig.add_scatter(x=data.eps, y=data.scalar_steps, name='Метод скалярных произведений')

    fig.update_xaxes(title='ε', type='log', autorange='reversed',tickformat='.0e')
    fig.update_yaxes(title='Количество шагов')

    fig.update_layout(title=f'Зависимость количества шагов от ε ({n = })')

    fig.show()

In [7]:
def compare_eps_and_value_error(data: pd.DataFrame, n: int, eps_boundary: Optional[float] = None) -> None:
    data = data[data.n == n]

    if eps_boundary is not None:
        data = data[data.eps <= eps_boundary]

    fig = go.Figure()

    fig.add_scatter(x=data.eps, y=data.power_value_error, name='Степенной метод')
    fig.add_scatter(x=data.eps, y=data.scalar_value_error, name='Метод скалярных произведений')

    fig.update_xaxes(title='ε', type='log', autorange='reversed', tickformat='.0e')
    fig.update_yaxes(title='Ошибка', tickformat='.2e')

    fig.update_layout(title=f'Зависимость ошибки вычисления собственного значения от ε ({n = })')

    fig.show()

In [8]:
def compare_eps_and_vector_error(data: pd.DataFrame, n: int, eps_boundary: Optional[float] = None) -> None:
    data = data[data.n == n]

    if eps_boundary is not None:
        data = data[data.eps <= eps_boundary]

    fig = go.Figure()

    fig.add_scatter(x=data.eps, y=data.power_vector_error, name='Степенной метод')
    fig.add_scatter(x=data.eps, y=data.scalar_vector_error, name='Метод скалярных произведений')

    fig.update_xaxes(title='ε', type='log', autorange='reversed', tickformat='.0e')
    fig.update_yaxes(title='Ошибка', tickformat='.2e')

    fig.update_layout(title=f'Зависимость ошибки вычисления собственного вектора от ε ({n = })')

    fig.show()

# Diagonal

In [9]:
def get_diagonal_matrix(size: int) -> np.ndarray:
    return np.diag(range(1, size + 1))

In [10]:
diagonal_data = pd.DataFrame(product(range(2, 101), [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
diagonal_data[
    [
        'power_steps', 
        'power_value_error', 
        'power_vector_error', 
        'scalar_steps', 
        'scalar_value_error', 
        'scalar_vector_error'
    ]
] = diagonal_data.apply(
    func=lambda row: benchmark_matrix(get_diagonal_matrix(int(row.n)), row.eps),
    axis=1,
)
diagonal_data

,n,eps,power_steps,power_value_error,power_vector_error,scalar_steps,scalar_value_error,scalar_vector_error
0,2,1.000000e-02,8.0,4.577410e-05,3.906228e-03,4.0,1.949318e-03,0.031239
1,2,1.000000e-03,11.0,7.152552e-07,4.882812e-04,6.0,1.220554e-04,0.007812
2,2,1.000000e-04,15.0,2.793968e-09,3.051758e-05,7.0,3.051665e-05,0.003906
3,2,1.000000e-05,18.0,4.365575e-11,3.814697e-06,9.0,1.907345e-06,0.000977
4,2,1.000000e-06,21.0,6.821210e-13,4.768372e-07,11.0,1.192093e-07,0.000244
...,...,...,...,...,...,...,...,...
1381,100,1.000000e-11,2522.0,0.000000e+00,9.816593e-12,1066.0,4.896350e-10,0.000022
1382,100,1.000000e-12,2751.0,0.000000e+00,9.826986e-13,1181.0,4.853007e-11,0.000007
1383,100,1.000000e-13,2980.0,0.000000e+00,9.837390e-14,1286.0,5.883294e-12,0.000002
1384,100,1.000000e-14,3209.0,0.000000e+00,9.847806e-15,1363.0,1.250555e-12,0.000001


In [11]:
compare_eps_and_steps(diagonal_data, 10)
compare_eps_and_steps(diagonal_data, 50)
compare_eps_and_steps(diagonal_data, 100)

Unsupported

In [46]:
eps_boundary = 1e-8

compare_eps_and_value_error(diagonal_data, 10, eps_boundary)
compare_eps_and_value_error(diagonal_data, 50, eps_boundary)
compare_eps_and_value_error(diagonal_data, 100, eps_boundary)

Unsupported

In [47]:
eps_boundary = 1e-8

compare_eps_and_vector_error(diagonal_data, 10, eps_boundary)
compare_eps_and_vector_error(diagonal_data, 50, eps_boundary)
compare_eps_and_vector_error(diagonal_data, 100, eps_boundary)

Unsupported

# Diagonally dominant

In [14]:
def get_diagonally_dominant_matrix(size: int) -> np.ndarray:
    return np.eye(size) * 4 + np.eye(size, k=1) * -1 + np.eye(size, k=-1) * -1

In [15]:
diagonally_dominant_data = pd.DataFrame(product(range(2, 31), [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
diagonally_dominant_data[
    [
        'power_steps', 
        'power_value_error', 
        'power_vector_error', 
        'scalar_steps', 
        'scalar_value_error', 
        'scalar_vector_error'
    ]
] = diagonally_dominant_data.apply(
    func=lambda row: benchmark_matrix(get_diagonally_dominant_matrix(int(row.n)), row.eps),
    axis=1,
)
diagonally_dominant_data

,n,eps,power_steps,power_value_error,power_vector_error,scalar_steps,scalar_value_error,scalar_vector_error
0,2,1.000000e-02,1.0,2.000000e+00,1.570092e-16,1.0,2.000000,0.000000
1,2,1.000000e-03,1.0,2.000000e+00,1.570092e-16,1.0,2.000000,0.000000
2,2,1.000000e-04,1.0,2.000000e+00,1.570092e-16,1.0,2.000000,0.000000
3,2,1.000000e-05,1.0,2.000000e+00,1.570092e-16,1.0,2.000000,0.000000
4,2,1.000000e-06,1.0,2.000000e+00,1.570092e-16,1.0,2.000000,0.000000
...,...,...,...,...,...,...,...,...
401,30,1.000000e-11,1166.0,3.067876e-02,5.478716e-01,522.0,0.030679,0.547899
402,30,1.000000e-12,1278.0,3.067876e-02,5.478716e-01,578.0,0.030679,0.547880
403,30,1.000000e-13,1390.0,3.067876e-02,5.478716e-01,635.0,0.030679,0.547874
404,30,1.000000e-14,1503.0,3.067876e-02,5.478716e-01,687.0,0.030679,0.547873


In [16]:
compare_eps_and_steps(diagonally_dominant_data, 7)
compare_eps_and_steps(diagonally_dominant_data, 25)

Unsupported

In [48]:
eps_boundary = 1e-8

compare_eps_and_value_error(diagonally_dominant_data, 7, eps_boundary)
compare_eps_and_value_error(diagonally_dominant_data, 25, eps_boundary)

Unsupported

In [49]:
eps_boundary = 1e-8

compare_eps_and_vector_error(diagonally_dominant_data, 7, eps_boundary)
compare_eps_and_vector_error(diagonally_dominant_data, 25, eps_boundary)

Unsupported

# Random

In [19]:
random_matrix = np.array([
    [-1.19799, 0.27127, -0.87905, 1.64268, 1.40971, 0.43877, 0.33667, -0.44129, 0.28644, 0.15796],
    [0.59457, 1.02181, -1.74053, 0.20298, 0.8621, 0.88291, 0.66765, 0.36682, -1.39194, -3.04015],
    [-0.57733, 0.89507, 1.66682, -1.66866, -0.33356, 0.45389, -0.8239, 1.31179, 0.08234, 0.11522],
    [-0.62787, 0.02007, -1.80254, 0.48282, 0.73049, -1.65855, 0.17265, 0.62828, 1.00379, 0.15821],
    [-0.90439, 0.04459, -0.28638, -0.35911, -1.81614, -1.15808, -0.9615, 0.41279, 0.19879, 0.15275],
    [-0.29761, 0.49338, 0.17208, -1.56132, -0.44397, 0.63476, 1.55016, -1.41762, 0.71268, -1.0107],
    [-0.54966, -0.13497, 1.8501, -1.59812, 0.7826, 0.6689, 1.10238, -0.54145, -0.76669, -0.49043],
    [0.81368, 0.59331, 1.16274, -0.34751, 0.55001, -0.43039, 0.47586, -0.72489, 0.22, 0.01402],
    [0.74463, -2.08764, -0.40422, -0.08962, -1.30085, 0.12947, 1.3579, -0.48038, 0.75092, -1.35812],
    [0.43595, 0.9277, 0.34069, -1.58998, -1.20261, -0.31744, 0.23336, 0.71316, -0.68451, 0.0356],
])

In [20]:
random_data = pd.DataFrame(product([10], [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
random_data[
    [
        'power_steps', 
        'power_value_error', 
        'power_vector_error', 
        'scalar_steps', 
        'scalar_value_error', 
        'scalar_vector_error'
    ]
] = random_data.apply(
    func=lambda row: benchmark_matrix(random_matrix, row.eps),
    axis=1,
)
random_data

,n,eps,power_steps,power_value_error,power_vector_error,scalar_steps,scalar_value_error,scalar_vector_error
0,10,1.000000e-02,197.0,5.856740e-04,1.434672e-03,83.0,5.840821e-02,2.758082e-01
1,10,1.000000e-03,245.0,6.802123e-05,1.663461e-04,107.0,6.313106e-03,9.464796e-02
2,10,1.000000e-04,297.0,1.042036e-06,1.480572e-05,131.0,6.922654e-04,3.068941e-02
3,10,1.000000e-05,345.0,8.031842e-08,1.635557e-06,155.0,7.577581e-05,1.068008e-02
4,10,1.000000e-06,397.0,8.074326e-08,1.490928e-07,179.0,8.294415e-06,3.486558e-03
5,10,1.000000e-07,446.0,1.380063e-08,1.705739e-08,179.0,8.294415e-06,3.486558e-03
6,10,1.000000e-08,497.0,1.483285e-09,1.539994e-09,227.0,9.932015e-08,3.852489e-04
7,10,1.000000e-09,546.0,6.824230e-11,1.655767e-10,251.0,1.086524e-08,1.293379e-04
8,10,1.000000e-10,598.0,9.858780e-13,1.473538e-11,275.0,1.188384e-09,4.261055e-05
9,10,1.000000e-11,646.0,7.283063e-14,1.627937e-12,312.0,4.336309e-11,7.933457e-06


In [21]:
compare_eps_and_steps(random_data, 10)

Unsupported

In [50]:
compare_eps_and_value_error(random_data, n=10, eps_boundary=1e-8)

Unsupported

In [51]:
compare_eps_and_vector_error(random_data, n=10, eps_boundary=1e-8)

Unsupported

# Hilbert

In [24]:
hilbert_data = pd.DataFrame(product(range(2, 29), [10 ** -i for i in range(2, 16)]), columns=['n', 'eps'])
hilbert_data[
    [
        'power_steps', 
        'power_value_error', 
        'power_vector_error', 
        'scalar_steps', 
        'scalar_value_error', 
        'scalar_vector_error'
    ]
] = hilbert_data.apply(
    func=lambda row: benchmark_matrix(get_hilbert_matrix(int(row.n)), row.eps),
    axis=1,
)
hilbert_data

,n,eps,power_steps,power_value_error,power_vector_error,scalar_steps,scalar_value_error,scalar_vector_error
0,2,1.000000e-02,3.0,4.192381e-07,4.223772e-05,1.0,1.536973e-05,8.144052e-04
1,2,1.000000e-03,3.0,4.192381e-07,4.223772e-05,2.0,4.134201e-08,4.223772e-05
2,2,1.000000e-04,4.0,1.127667e-09,2.190586e-06,2.0,4.134201e-08,4.223772e-05
3,2,1.000000e-05,5.0,3.033129e-12,1.136110e-07,3.0,1.112017e-10,2.190586e-06
4,2,1.000000e-06,6.0,8.215650e-15,5.892235e-09,3.0,1.112017e-10,2.190586e-06
...,...,...,...,...,...,...,...,...
373,28,1.000000e-11,22.0,1.332268e-15,6.316588e-13,11.0,1.882938e-13,1.931444e-07
374,28,1.000000e-12,23.0,1.332268e-15,1.788920e-13,12.0,1.754152e-14,5.461476e-08
375,28,1.000000e-13,25.0,1.332268e-15,1.469203e-14,13.0,3.108624e-15,1.544322e-08
376,28,1.000000e-14,27.0,1.554312e-15,1.724367e-15,14.0,1.776357e-15,4.366825e-09


In [25]:
compare_eps_and_steps(hilbert_data, 2)
compare_eps_and_steps(hilbert_data, 14)
compare_eps_and_steps(hilbert_data, 28)

Unsupported

In [52]:
eps_boundary = 1e-8

compare_eps_and_value_error(hilbert_data, 2, eps_boundary)
compare_eps_and_value_error(hilbert_data, 14, eps_boundary)
compare_eps_and_value_error(hilbert_data, 28, eps_boundary)

Unsupported

In [53]:
eps_boundary = 1e-8

compare_eps_and_vector_error(hilbert_data, 2, eps_boundary)
compare_eps_and_vector_error(hilbert_data, 14, eps_boundary)
compare_eps_and_vector_error(hilbert_data, 28, eps_boundary)

Unsupported